In [1]:
!pip install crewai crewai-tools langchain openai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.4/80.4 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of instructor to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 5.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opentelemetry-exporter-otlp-proto-grpc to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 666.8/666.8 kB 24.3

In [2]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
# Optional: If you’ll use web search tools
os.environ["SERPER_API_KEY"] = userdata.get("SERPER_API_KEY")
os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"

In [3]:
import crewai
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool

In [4]:
print(crewai.__version__)

1.7.2


Initializing the tool

In [ ]:
#search_tool = SerperDevTool()

In [5]:
from crewai_tools import YoutubeChannelSearchTool
yt_tool = YoutubeChannelSearchTool()
# The YoutubeChannelSearchTool is currently experiencing issues with parsing YouTube channel handles.
# It is likely due to changes in YouTube's website structure or API, affecting the underlying 'pytube' library.
# Therefore, this tool cannot be initialized successfully at this time.
# yt_tool = YoutubeChannelSearchTool(youtube_channel_handle="sanesacademyofstatistics4607")
# To prevent further errors, the initialization of yt_tool has been commented out.

Defining the Agent

In [6]:
blog_researcher = Agent(
    role='Blog Researcher from YouTube videos',
    goal="Get the relevant video content for the topic {topic} from YouTube channel",
    backstory='You are an expert in understanding the videos on A.I., Data Science, Machine Learning and allied topics',
    tools=[yt_tool],
    llm_name="gpt-4o-mini",
    verbose=True,
    memory=True,
    allow_delegation=True
)

In [8]:
blog_writer = Agent(
    role='Writer',
    goal="Narrate compelling tech stories about the video {topic} from YouTube channel",
    backstory=(
        "With a flair for simplifying complex tipics, you craft"
        "engagaing narratives that captivate and educate, bringing new"
        "discoveries to light in an accessible manner."
        ),
    tools=[yt_tool],
    llm_name="gpt-4o-mini",
    verbose=True,
    memory=True,
    allow_delegation=False
)

Defining a task

In [9]:
research_task = Task(
    description=(
        "Identify the video {topic}."
        "Get the detailed information about the video from the channel."
    ),
    expected_output="A comprehensive 3-paragraph long report based on the {topic} of the video content.",
    tools=[yt_tool],
    agent=blog_researcher
)

In [10]:
write_task = Task(
    description=(
        "get the info from the YouTube channel on the {topic}."
    ),
    expected_output="Summarize the info from the YouTube channel video on the {topic} and create the content from the blog.",
    tools=[yt_tool],
    async_execution=False,
    agent=blog_writer,
    output_file="my-blog-post.md"
)

Building the crew

In [11]:
from crewai import llm
crew = Crew(
    agents=[blog_researcher, blog_writer],
    tasks=[research_task, write_task],
    process=Process.sequential,
    memory=True,
    cache=True,
    max_rpm=100,
    share_crew=True
)

Running the crew

In [12]:
result = crew.kickoff(inputs={'topic':'#1 Machine Learning Engineering for Production (MLOps) Specialization [Course 1, Week 1, Lesson 1]',
                              "youtube_channel_handle": "@Deeplearningai"})
print(result)

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Blog Researcher from YouTube videos                                                                     │
│                                                                                                                 │
│  Task: Identify the video #1 Machine Learning Engineering for Production (MLOps) Specialization [Course 1,      │
│  Week 1, Lesson 1].Get the detailed information about the video from the channel.                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯



I encountered an error while trying to use the tool. This was the error: Invalid YouTube channel URL: @youtube_channel_handle_here.
 Tool Search a Youtube Channels content accepts these inputs: Tool Name: Search a Youtube Channels content
Tool Arguments: {
  "description": "Input for YoutubeChannelSearchTool.",
  "properties": {
    "search_query": {
      "description": "Mandatory search query you want to use to search the Youtube Channels content",
      "title": "Search Query",
      "type": "string"
    },
    "youtube_channel_handle": {
      "description": "Mandatory youtube_channel_handle path you want to search",
      "title": "Youtube Channel Handle",
      "type": "string"
    }
  },
  "required": [
    "search_query",
    "youtube_channel_handle"
  ],
  "title": "YoutubeChannelSearchToolSchema",
  "type": "object",
  "additionalProperties": false
}
Tool Description: A tool that can be used to semantic search a query from a Youtube Channels content.



╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Blog Researcher from YouTube videos                                                                     │
│                                                                                                                 │
│  Thought: Action: Search a Youtube Channels content                                                             │
│                                                                                                                 │
│  Using Tool: Search a Youtube Channels content                                                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  {                                                                                                              │
│    "search_query": "Machine Learning Engineering for Production (MLOps) Specialization Course 1 Week 1 Lesson   │
│  1",                                                                                                            │
│    "youtube_channel_handle": "youtube_channel_handle_here"                                                      │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│  I encountered an error while trying to use the tool. This was the error: Invalid YouTube channel URL:          │
│  @youtube_channel_handle_here.                                                                                  │
│   Tool Search a Youtube Channels content accepts these inputs: Tool Name: Search a Youtube Channels content     │
│  Tool Arguments: {                                                                                              │
│    "description": "Input for YoutubeChannelSearchTool.",                                                        │
│    "properties": {                                                                                              │
│      "search_query": {                                                                                          │
│        "description": "Mandatory search query you want to use to search the Youtube Channels content",          │
│        "title": "Search Query",                                                                                 │
│        "type": "string"                                                                                         │
│      },                                                                                                         │
│      "youtube_channel_handle": {                                                                                │
│        "description": "Mandatory youtube_channel_handle path you want to search",                               │
│        "title": "Youtube Channel Handle",                                                                       │
│        "type": "string"                                                                                         │
│      }                                                                                                          │
│    },                                                                                                           │
│    "required": [                                                                                                │
│      "search_query",                                                                                            │
│      "youtube_channel_handle"                                                                                   │
│    ],                                                                                                           │
│    "title": "YoutubeChannelSearchToolSchema",                                                                   │
│    "type": "object",                                                                                            │
│    "additionalProperties": false                                                                                │
│  }                                                                                                              │
│  Tool Description: A tool that can be used to semantic search a query from a Youtube Channels content..         │
│  Moving on then. I MUST either use a tool (use one at time) OR give my best final answer not both at the same   │
│  time. When responding, I must use the following format:                                                        │
│                                                                                                                 │
│  ```                                                                                                            │
│  Thought: you should always think about what to do                                                              │
│  Action: the action to take, should be one of [Search 

...



I encountered an error while trying to use the tool. This was the error: Invalid YouTube channel URL: @AnalyticsVidhya.
 Tool Search a Youtube Channels content accepts these inputs: Tool Name: Search a Youtube Channels content
Tool Arguments: {
  "description": "Input for YoutubeChannelSearchTool.",
  "properties": {
    "search_query": {
      "description": "Mandatory search query you want to use to search the Youtube Channels content",
      "title": "Search Query",
      "type": "string"
    },
    "youtube_channel_handle": {
      "description": "Mandatory youtube_channel_handle path you want to search",
      "title": "Youtube Channel Handle",
      "type": "string"
    }
  },
  "required": [
    "search_query",
    "youtube_channel_handle"
  ],
  "title": "YoutubeChannelSearchToolSchema",
  "type": "object",
  "additionalProperties": false
}
Tool Description: A tool that can be used to semantic search a query from a Youtube Channels content.



╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Writer                                                                                                  │
│                                                                                                                 │
│  Thought: Thought: I need to continue searching with the aim of finding a satisfactory channel handle for the   │
│  Machine Learning Engineering for Production (MLOps) Specialization content. I will shift to using a broader    │
│  educational handle known for delivering comprehensive content.                                                 │
│                                                                                                                 │
│  Using Tool: Search a Youtube Channels content                                                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  {                                                                                                              │
│    "search_query": "#1 Machine Learning Engineering for Production (MLOps) Specialization [Course 1, Week 1,    │
│  Lesson 1]",                                                                                                    │
│    "youtube_channel_handle": "AnalyticsVidhya"                                                                  │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│  I encountered an error while trying to use the tool. This was the error: Invalid YouTube channel URL:          │
│  @AnalyticsVidhya.                                                                                              │
│   Tool Search a Youtube Channels content accepts these inputs: Tool Name: Search a Youtube Channels content     │
│  Tool Arguments: {                                                                                              │
│    "description": "Input for YoutubeChannelSearchTool.",                                                        │
│    "properties": {                                                                                              │
│      "search_query": {                                                                                          │
│        "description": "Mandatory search query you want to use to search the Youtube Channels content",          │
│        "title": "Search Query",                                                                                 │
│        "type": "string"                                                                                         │
│      },                                                                                                         │
│      "youtube_channel_handle": {                                                                                │
│        "description": "Mandatory youtube_channel_handle path you want to search",                               │
│        "title": "Youtube Channel Handle",                                                                       │
│        "type": "string"                                                                                         │
│      }                                                                                                          │
│    },                                                                                                           │
│    "required": [                                                                                                │
│      "search_query",                                                                                            │
│      "youtube_channel_handle"                                                                                   │
│    ],                                                                                                           │
│    "title": "YoutubeChannelSearchToolSchema",                                                                   │
│    "type": "object",                                                                                            │
│    "additionalProperties": false                                                                                │
│  }                                                                                                              │
│  Tool Description: A tool that can be used to semantic search a query from a Youtube Channels content..         │
│  Moving on then. I MUST either use a tool (use one at time) OR give my best final answer not both at the same   │
│  time. When responding, I must use the following format:                                                        │
│                                                                                                                 │
│  ```                                                                                                            │
│  Thought: you should always think about what to do                                                              │
│  Action: the action to take, should be one of [Search 

KeyboardInterrupt: 